<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/32b_langchain_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now we will begin exploring Conversational Memory for LLMs using Langchain.


# 1. Import Library

Install OpenAI and LangChain Library by Uncomment Cell below and Run (Shift + Enter)

In [5]:
# ! pip install langchain


Import the LangChain Library

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 2. Authentication

Before we can access and utilize ChatGPT and GPT-4 from OpenAI, we need to register, sign up, and provide the necessary payment information.

## Sign up

Click on the green Sign up button in the link below to register for OpenAI.

https://platform.openai.com/overview

Please enter your credit card information on the Billing page.

https://platform.openai.com/account/billing/payment-methods






## API Key

To generate an API Key, please follow the steps below:

1. Access the link provided below.
2. Click on the "Create new secret key" button.
3. Give a name to the key.
4. Copy the secret key generated and keep it ready to be entered in the "OPENAI_API_KEY" field below.

Link: https://platform.openai.com/account/api-keys

Please make sure to securely store and protect your API Key as it grants access to the OpenAI API.


## Sample API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

Please execute the cell below and enter the secret key.


In [3]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

os.environ["OPENAI_API_KEY"] = token

Enter OPENAI_API_KEY here: ··········


After completing the testing, it is recommended to go back and delete the Secret Key on the page https://platform.openai.com/account/api-keys every time for security purposes. We can generate new keys continuously and without limitations.

#3. Chatbot

## ConversationBufferMemory

Note that verbose=True

In [8]:
chatllm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chatllm, 
    memory = memory,
    verbose=True
)

In [9]:
conversation.predict(input="Hi, my name is Ken")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Ken
AI:

> Finished chain.


"Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?"

In [10]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ken
AI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [11]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ken
AI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Ken, as you mentioned earlier.'

In [12]:
print(memory.buffer)

Human: Hi, my name is Ken
AI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Ken, as you mentioned earlier.


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Ken\nAI: Hello Ken, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Ken, as you mentioned earlier."}

In [14]:
memory = ConversationBufferMemory()

In [15]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [16]:
print(memory.buffer)

Human: Hi
AI: What's up


In [17]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [18]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [19]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}